In [2]:
from __future__ import unicode_literals
from xml.etree import ElementTree as Xet
import csv
import pandas as pd
import numpy as np
import urllib3
import requests
#%pip install requests beautifulsoup4
#%pip install lxml
import bs4
from bs4 import BeautifulSoup as bs

#https://github.com/LaurenceRawlings/savify
#%pip install -U savify
from savify import Savify
from savify.types import Type, Format, Quality
from savify.utils import PathHolder
from savify.logger import Logger

#https://github.com/alexmercerind/youtube-search-python
#%pip install youtube-search-python
#import youtubesearchpython as yts

#https://github.com/ytdl-org/youtube-dl#embedding-youtube-dl
#https://stackoverflow.com/questions/32482230/how-to-set-up-default-download-location-in-youtube-dl
import youtube_dl
import os

#unused
#%pip install music-tag
#%pip install pytube
#from pytube import YouTube

In [3]:
# Module 1: Playing with html pages
# Converts the xml page into a dataframe, filters based on watching/completed and also adds mal link in last column
def MakeCSV(exportedxml):
    xml = Xet.parse(exportedxml)
    csvfile = open('mal_to_csv.csv','w',encoding='utf-8')
    csvfile_writer = csv.writer(csvfile)
    csvfile_writer.writerow(['AnimeTitle','ID','Status'])

    for anime in xml.findall('anime'):
        if(anime):
            animename = anime.find('series_title')
            animeid = anime.find('series_animedb_id')
            animestatus = anime.find('my_status')
            csv_line = [animename.text, animeid.text, animestatus.text]
            csvfile_writer.writerow(csv_line)
    csvfile.close()  

    df = pd.read_csv('mal_to_csv.csv')
    df2= df.dropna()
    filters = ['Completed', 'Watching']
    df3 = df2[df2['Status'].isin(['Completed', 'Watching'])].reset_index()
    del df3['index']
    def make_url(row):
        return 'https://myanimelist.net/anime/' + str(row[1])
    df3['URL'] = df3.apply(make_url, axis = 1)
    return df3

# This function gives the title, season and year of the anime as a list
def AnimeAndSeason(entry_page):
    title  = entry_page.title.text.replace('\n', ' ').split(' - ')[0].strip()
    links  = entry_page.select('span > a')
    years  = range(1900,2050)
    season = ''
    year   = 1900
    for row in links:
        test = row.contents[0].split()
        if len(test) == 2 and test[1].isdigit() and int(test[1]) in years:
            season_info = test
            season = season_info[0] + ' ' + season_info[1]
            year = int(season_info[1])
    return(title, season, year)

# Function below gives a lil bit fucked list of the category/song/artist for individual mal page
def SongAndArtist(entry_page):
    testing = entry_page.find_all('div', attrs={'class':'di-tc va-t'})
    testing2 = entry_page.find_all('div', attrs={'class':'di-tc va-t borderDark pb4'})
    testing3 = testing + testing2
    SAAlist = []
    for entry in testing3:
        category = entry.h2
        if type(category) == bs4.element.Tag: 
            category_name = category.contents
            SAAlist.append(category_name)

        song_td_set = entry.find_all('td', {'width':'84%'})
        for song_td in song_td_set:
            song_data = song_td.text.replace('\n', ' ').strip()
            row = song_data.split('\xa0') #split song# and song-artist
            for datas in row:
                if ' by ' in datas:
                    row2 = datas.split(' by ') #split song and artist
                    data_entry = [row2[0].strip('\"'), row2[1]]  #join song, artist
                    song_links = song_td.find_all('input')
                    for link in song_links:
                        somelink = link['value']
                        if 'spotify' in somelink:
                            data_entry.append(somelink)
                    SAAlist.append(data_entry)
    return SAAlist

# This combines the previous 2 functions and gives a nice dataframe of each row having all the identifying factors of a song
def CreateSongDF(entry_page):
    aas = AnimeAndSeason(entry_page)
    saa = SongAndArtist(entry_page)
    if aas[0] == 'Gyo (GYO: Tokyo Fish Attack!)': saa[2][0] = 'End Theme'
    
    for line in saa:
        if 'Opening Theme' in line: op_index = saa.index(line)
        if 'Ending Theme' in line: ed_index = saa.index(line)
                   
    OP = []
    category = saa[op_index][0]
    for op in range(op_index+1,ed_index):
        temp = saa[op]
        temp.insert(0,category)
        OP.append(temp)

    ED = []
    category = saa[ed_index][0]
    for ed in range(ed_index+1,len(saa)):
        temp = saa[ed]
        temp.insert(0,category)
        ED.append(temp)
    
    def Clean_Blanks(List, Length):
        for item in List:
            if len(item) != Length:
                item.extend(' ')
       
    Clean_Blanks(ED, 4)
    Clean_Blanks(OP, 4)
    song_columns = ['category', 'songname', 'artist', 'link']
    opdb = pd.DataFrame(OP, columns = song_columns)
    eddb = pd.DataFrame(ED, columns = song_columns)
    SongDB = pd.concat([opdb,eddb], axis = 0, ignore_index = True)
    SongDB['status'] = ''
    return SongDB

# Module 2: Playing with internet for spotify and youtube links
# This creates a new dataframe (withc columns as song metadata and a 'links' column) from the csv containing mal urls
def GetSpotify(malcsv, start, length):
    song_table = pd.read_csv(malcsv).reset_index()
    song_table.drop(['Unnamed: 0', 'index'], inplace=True, axis=1)
    song_table = song_table.iloc[range(start, start+length), :]
    link_table = pd.DataFrame(columns = ['anime', 'season', 'year','category', 'songname', 'artist', 'link', 'status'])
    for i in song_table.index:
        anime_link = song_table.loc[i,'URL']
        html       = requests.get(anime_link).text #type: ignore       
        mal_page   = bs(html, 'lxml')  
        aas        = AnimeAndSeason(mal_page)

        song_subtable    = CreateSongDF(mal_page)
        anime_subtable   = pd.DataFrame(columns=['anime', 'season', 'year'])
        print(i, 'Fetching spotify links for', aas[0])

        for i in range(0, len(song_subtable)):
            anime_subtable.loc[i,'anime'], anime_subtable.loc[i,'season'],  anime_subtable.loc[i,'year']   = aas[0], aas[1], aas[2]
        anime_subtable = pd.concat([anime_subtable, song_subtable], axis = 1)
        anime_subtable.to_csv('sheet_w_spot.csv', mode = 'a', index = False, header = False)

# Function to make all entries in the databases valid filenames
def ValidFile(txt):
    filename = ''
    if '...' in txt:
        txt = txt.strip('...')
    for i in txt:
        if i not in '\/*?<>|"':
            filename = filename + i
        elif i in ':':
            filename = filename + '-'
        else:
            filename = filename + '_'
    return filename

# Module 3: Playing with external libraries to download songs
# Downloads spotify links into appropriate nested folder
def DownloadSpotify(sheet_with_spotify, dl_loc, start, end):
    songset = sheet_with_spotify.iloc[start:end]
    for i in songset.index:
        if 'spotify' not in songset.loc[i,'link'] or songset.loc[i,'status'] == 'Downloaded': continue
        #https://developer.spotify.com/dashboard/applications/f2d89a80d48e41e19e4263d5b3792c21 for api credentials
        sp_link = songset.loc[i,'link']
        year    = str(songset.loc[i,'year'])
        season  = str(songset.loc[i,'season'])
        oped    = songset.loc[i,'category']
        anime   = ValidFile(songset.loc[i,'anime'])
        title   = ValidFile(songset.loc[i,'songname'].strip())
        artist  = ValidFile(songset.loc[i,'artist'].strip())
        path    = year + '/' + season + '/' + anime + '/' + oped + '/' + artist + '/'+ title
        ffpath  = 'ffmpeg'
        dlpath  = dl_loc +'/' + 'Animusic'
        api1    = 'f2d89a80d48e41e19e4263d5b3792c21'
        api2    = '12f6688e93504599b3d0403e8a4f2cc8'
        logger  = Logger(log_location = dlpath, log_level = None)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print(i, path)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        
        #C:\Users\dell\AppData\Roaming\Savify\temp is where the temporary download goes
        s = Savify(api_credentials  = (api1,api2), 
                    download_format = Format.MP3, 
                    ffmpeg_location = ffpath, 
                    skip_cover_art  = False, 
                    path_holder = PathHolder(downloads_path=dlpath), 
                    group   = path, 
                    quality = Quality.BEST,
                    logger  = logger)
        s.download(sp_link)
        sheet_with_spotify.loc[i,'status'] = 'Downloaded'

# Downloads youtube links into appropriate nested folder 
def DownloadYoutube(sheet_with_youtube, dl_loc, start, end):
    songset2 = sheet_with_youtube.loc[start:end,:]
    for i in songset2.index:
        if 'youtube' not in songset2.loc[i,'link'] or str(songset2.loc[i,'status']) == 'Downloaded' or str(songset2.loc[i,'status']) == 'sus': continue
        year    = str(songset2.loc[i,'year'])
        season  = str(songset2.loc[i,'season'])
        anime   = songset2.loc[i,'anime']
        oped    = songset2.loc[i,'category']
        title   = songset2.loc[i,'songname']
        artist  = songset2.loc[i,'artist']
        yt_link = songset2.loc[i,'link']
        path    = dl_loc + '/' + year + '/' + season + '/' + anime + '/' + oped + '/' + artist + '/'+ title + '/'
        def my_hook(d):
            if d['status'] == 'finished':
                print('~~~~~~', i, title, artist,'~~~~~~')

        ydl_opts = {
            'format': 'bestaudio',
            'outtmpl': path + '%(title)s.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',}],
                'progress_hooks': [my_hook]}

        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([yt_link])
        sheet_with_youtube.loc[i,'status'] = 'Downloaded'
    return sheet_with_youtube

In [ ]:
## Converting xml to a csv with eligible anime and their mal links
MakeCSV('mal_export.xml').to_csv('mal_to_csv_filtered.csv')

In [ ]:
## Creating a new csv of all songs and spotify links, have to be done 150 at a time
#  Also some links are broken so you will have to manually fix it 
link_table = pd.DataFrame(columns = ['anime', 'season', 'year','category', 'songname', 'artist', 'link', 'status'])
GetSpotify('mal_to_csv_filtered.csv', 0, 150) # file, start, no of entries 

In [127]:
## Ensuring robustness for new database, i.e. everytime a new mal list is exported
#  If a song is in song db, it's already dled, so need a code that will filter existing songs from new db, and then append the new songs after dling
old = pd.read_csv('Older/sheet_d_spot_w_yt_copy.csv')
new = pd.read_csv('sheet_w_spot.csv')
new = new.replace(' ', pd.NA)
old.drop('extra', inplace = True, axis = 1)

#filtering duplicates in new df
new1 = new[new['link'].isna()]
new2 = new[new["link"] != ""].drop_duplicates(subset=["link"])
new3 = pd.concat([new1, new2], axis=0).sort_index()
new = new3.drop_duplicates(subset=["songname", "artist", "category"])

for index, row in new.iterrows():
    # Check if the combination of anime, songname, and artist match in the old DataFrame
    match_condition = (old['anime'] == row['anime']) & (old['songname'] == row['songname']) & (old['artist'] == row['artist'])

    # Check if the link is different in the new DataFrame
    if match_condition.any() and pd.isnull(row['link']) == 0 and old.loc[match_condition, 'link'].values[0] != row['link']:
        # Update the link in the old DataFrame
        old.loc[match_condition, 'link'] = row['link']

        # Check if the status column in the old DataFrame contains "Downloaded" and replace it with blank
        if old.loc[match_condition, 'status'].values[0] == "Downloaded":
            old.loc[match_condition, 'status'] = 'SEX'

del old['Unnamed: 0']
old.to_csv('Wecooked.csv')
#https://open.spotify.com/track/47mtXCSExRrKA9cYGuiftZ


In [125]:
pd.set_option('display.max_rows', 100)
old.head(50)


In [ ]:
## Downloading songs from spotify
#  Ensure you've added all new links manually as well. 
dl_spot = pd.read_csv('sheet_w_spot.csv')
#dl_spot = pd.read_csv('sheet_d_spot_w_yt.csv') #uncomment for reruns
del dl_spot['Unnamed: 0']
 
DownloadSpotify(dl_spot, 'D:/_Animusic/', 0, 0)
dl_spot.to_csv('sheet_d_spot_w_yt.csv')

In [ ]:
## Downloading all songs from youtube links
yt = pd.read_csv('sheet_d_spot_s_yt.csv')
#dl_spot = pd.read_csv('sheet_d_spot_d_yt.csv') #uncomment for reruns
del yt['Unnamed: 0']
DownloadYoutube(yt, 'F:/Animusic', 0, 50)
yt.to_csv('sheet_d_spot_d_yt.csv')



In [128]:
#pokefarm 

from bs4 import BeautifulSoup
import pandas as pd
import datetime

# Initialize a dictionary to store user scores
user_scores = {}

# Function to scrape data and append to CSV
def scrape_and_append_to_csv():
    payload = {'inUserName': 'tanmayonnaise', 'inUserPass': '534519B4C10C03F2'}
    url = 'https://pokefarm.com/stats/interactions'
    html = requests.get(url, data=payload) #type: ignore

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html.content, 'html.parser')

    # Find all the <li> elements within the <ol> with class "leaderboard"
    leaderboard_items = soup.find_all('li')

    # Get the current timestamp
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Loop through the <li> elements to update user scores
    for item in leaderboard_items:
        # Extract the user's name from the <a> element, accounting for both classes
        user_link = item.find('a', class_='userlink0') or item.find('a', class_='userlink8')
        user = user_link.text.strip() if user_link else ""

        # Extract the user's score from the <span> element
        score = item.find('span', class_='score').text.strip()

        # Update the user's score in the dictionary
        user_scores[user] = score

    # Append the timestamp as a new column
    user_scores["Timestamp"] = timestamp

    # Create a DataFrame from the dictionary
    df = pd.DataFrame([user_scores])

    # Define the CSV file name
    csv_file_name = "leaderboard_data.csv"

    # Append the data to the existing CSV file, or create a new file if it doesn't exist
    df.to_csv(csv_file_name, mode='a', header=False, index=False)
    
    print(f"Leaderboard data has been appended to {csv_file_name} at {timestamp}.")

# Run the code to scrape and append data to CSV
scrape_and_append_to_csv()


Leaderboard data has been appended to leaderboard_data.csv at 2023-10-04 10:42:24.
